# COMP9517: Computer Vision 2024 T3 Project Method: UNet

## Our Unet and UDTransNet are using the same pipeline.
## Just Choose the UNet in the `Config.py` file.
```python
# Choose the model
# model_name = 'UDTransNet'
model_name = 'UNet'
```

## After the model change, the training and testing process is the same as UDTransNet.
## So, feel free to jump to Step 6.





### Step 1: Clone the UDTransNet model from github

### We recommend running this project on Google Colab.

### The pretrained three model weights using UNet can be accessed by using this [Google Drive link](https://drive.google.com/drive/folders/1qb72K2x3251G2_OJTlKDhs7qQBlkRzQD?usp=drive_link).

In [ ]:
!git clone https://github.com/McGregorWwww/UDTransNet.git

### Step 2: Install the required packages.


In [ ]:
%cd UDTransNet

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install tensorboardX
!pip install ml_collections

### Step 3: Download the SeaTurtleID2022 dataset from kaggle

In [ ]:
!curl -L -o ./archive.zip https://www.kaggle.com/api/v1/datasets/download/wildlifedatasets/seaturtleid2022

In [ ]:
!unzip archive.zip

### Step 4: Generate masks of the dataset with areas in ['turtle', 'head', 'flippers']

### In this method, I need to partition the turtle into three distinct areas to perform binary segmentation over each area.


In [ ]:
%mv turtles-data/ ./datasets/
%cd ./datasets/turtles-data/data

In [ ]:
# Generate masked images with areas

from pycocotools.coco import COCO
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd

# Choose one area to generate
# area = 'turtle'
area = 'head'
# area = 'flipper'

image_dir = ''
annotation_file = './annotations.json'
metadata_file = './metadata_splits.csv'
train_valid_folder_mask = f'./{area}_train_valid_folder_mask'
test_folder_mask = f'./{area}_test_folder_mask'

# Read the COCO styled annotation file
coco = COCO(annotation_file)

metadata = pd.read_csv(metadata_file)

# Get one category's ids e.g. "turtle"
category_ids = coco.getCatIds(catNms=[area])
image_ids = coco.getImgIds(catIds=category_ids)

Path(train_valid_folder_mask).mkdir(parents=True, exist_ok=True)
Path(test_folder_mask).mkdir(parents=True, exist_ok=True)

# 
def generate_and_save_mask(image_id):
    img_info = coco.loadImgs(image_id)[0]
    image_path = os.path.join(image_dir, img_info['file_name'])

    image = Image.open(image_path)
    image = np.array(image)

    ann_ids = coco.getAnnIds(imgIds=img_info['id'], catIds=category_ids)
    anns = coco.loadAnns(ann_ids)

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in anns:
        mask = np.maximum(mask, coco.annToMask(ann))

    split_label = metadata.loc[metadata['file_name'] == img_info['file_name'], 'split_closed'].values[0]

    mask_image = Image.fromarray(mask * 255)
    if split_label in ['train', 'valid']:
        mask_image.save(os.path.join(train_valid_folder_mask, f"{img_info['file_name'].split('/')[-1].split('.')[0]}_mask.png"))
    elif split_label == 'test':
        mask_image.save(os.path.join(test_folder_mask, f"{img_info['file_name'].split('/')[-1].split('.')[0]}_mask.png"))

for image_id in tqdm(image_ids, desc="Generating masks"):
    generate_and_save_mask(image_id)


### Step 5: Divide all images into a training folder and a testing folder based on the 'split_closed' column in the CSV file.


In [ ]:
from pycocotools.coco import COCO
from pathlib import Path
import pandas as pd
import os
from tqdm import tqdm

# area = 'turtle'
area = 'head'
# area = 'flipper'

image_dir = '/content/UDTransNet/datasets/turtles-data/data'
annotation_file = '/content/UDTransNet/datasets/turtles-data/data/annotations.json'
metadata_file = '/content/UDTransNet/datasets/turtles-data/data/metadata_splits.csv'
train_valid_folder = f'/content/UDTransNet/datasets/turtles-data/data/{area}_train_valid_folder'
test_folder = f'/content/UDTransNet/datasets/turtles-data/data/{area}_test_folder'

coco = COCO(annotation_file)

metadata = pd.read_csv(metadata_file)

# Get one category's ids e.g. "turtle"
category_ids = coco.getCatIds(catNms=[area])
image_ids = coco.getImgIds(catIds=category_ids)

Path(train_valid_folder).mkdir(parents=True, exist_ok=True)
Path(test_folder).mkdir(parents=True, exist_ok=True)

# Go through all the ids of the image and split them into training folder or testing folder base on the split label
for image_id in tqdm(image_ids, desc="Processing images"):
    img_info = coco.loadImgs(image_id)[0]
    image_path = os.path.join(image_dir, img_info['file_name'])

    if not os.path.exists(image_path):
        print(f"Image {image_path} not found. Skipping...")
        continue

    # Get all the annotation ids
    ann_ids = coco.getAnnIds(imgIds=img_info['id'])
    anns = coco.loadAnns(ann_ids)
    
    # Check if the area was in the annotations
    if len(anns) > 0:
        # Get split label from the matadata_split.csv
        split_label = metadata.loc[metadata['file_name'] == img_info['file_name'], 'split_closed'].values

        if len(split_label) == 0:
            print(f"No metadata found for image {img_info['file_name']}. Skipping...")
            continue

        split_label = split_label[0]
        if split_label in ['train', 'valid']:
            target_folder = train_valid_folder
        elif split_label == 'test':
            target_folder = test_folder
        else:
            continue

        target_path = os.path.join(target_folder, os.path.basename(image_path))
        try:
            Path(target_path).write_bytes(Path(image_path).read_bytes())
            print(f"Copied {image_path} to {target_folder}")
        except Exception as e:
            print(f"Error copying {image_path}: {e}")
    else:
        print(f"Image {img_info['file_name']} does not contain '{area}' annotation. Skipping...")

### Step 6: Create training and testing folders for the models

- The “img” folder in both training and testing folders is intended to store original images.
- The “labelcol” folder in both training and testing folders is intended to store masked ground truth images.

Then prepare the datasets in the following format for easy use of the code:
```text
├── datasets
│   ├── Turtle
│   │   ├── Test_Folder
│   │   │   ├── img
│   │   │   └── labelcol
│   │   └── Train_Folder
│   │       ├── img
│   │       └── labelcol
│   ├── Turtle_H
│   │   ├── Test_Folder
│   │   │   ├── img
│   │   │   └── labelcol
│   │   └── Train_Folder
│   │       ├── img
│   │       └── labelcol
│   └── Turtle_F
│       ├── Test_Folder
│       │   ├── img
│       │   └── labelcol
│       └── Train_Folder
│           ├── img
│           └── labelcol
```


### Move the split images into the corresponding folders as specified above


In [ ]:
%cd datasets
%mkdir Turtle
%cd Turtle
%mkdir Test_Folder Train_Folder
%mkdir Test_Folder/img Test_Folder/labelcol
%mkdir Train_Folder/img Train_Folder/labelcol

In [ ]:
%cd /content/UDTransNet/datasets
%mkdir Turtle_H
%cd Turtle_H
%mkdir Test_Folder Train_Folder
%mkdir Test_Folder/img Test_Folder/labelcol
%mkdir Train_Folder/img Train_Folder/labelcol

In [ ]:
%cd /content/UDTransNet/datasets
%mkdir Turtle_F
%cd Turtle_F
%mkdir Test_Folder Train_Folder
%mkdir Test_Folder/img Test_Folder/labelcol
%mkdir Train_Folder/img Train_Folder/labelcol

### Step 7: Start the training

## IF YOU WANT TO TRAIN IT YOURSELF, PLEASE REPLACE THE `Config.py`, `Load_Dataset.py`, `test_kfold.py` AND `train_kfold.py` FILES IN THE CLONED PROJECT WITH OURS.

## ⚠️Please remember to rename the original image folder that was split into two parts to “img,” and rename the ground truth image (mask) to “labelcol.”⚠️

## ⚠️You must organize them in the structure showed in Step 6!⚠️

### The first step is to change the settings in `Config.py`, all the configurations including learning rate, batch size and etc. are in it.

We optimize the convolution parameters in U-Net and the DAT parameters together with a single loss. Run:

```bash
python train_kfold.py
```

The results including log files, model weights, etc., are in '[TaskName]_kfold' folder, e.g., 'Turtle_kfold'.

In [ ]:
!python train_kfold.py

### Step 8: Start Testing

### !! First, change the session name in `Config.py` as the training phase. !!

### IF YOU ARE USING OUR PRE-TRAINED WEIGHT. 
### PUT THE MODEL TRAINED FOR `FLIPPERS` IN `Turtle_F_kfold/UDTransNet/Test_session_10.27_04h15/models/fold_1`
### PLEASE USE 'Test_session_10.27_04h15' for Flippers
### PUT THE MODEL TRAINED FOR `HEAD` IN `Turtle_H_kfold/UDTransNet/Test_session_10.27_03h17/models/fold_1`
### PLEASE USE 'Test_session_10.27_03h17' for Head
### PUT THE MODEL TRAINED FOR `CARAPACE` IN `Turtle_kfold/UDTransNet/Test_session_10.27_02h23/models/fold_1`
### PLEASE USE 'Test_session_10.27_02h23' for Carapace

Then, for Turtle, Turtle_H and Turtle_F, run:

```bash
python test_kfold.py
```

### You can get the Dice and IoU scores and the visualization results `in the visualized folder`

In [ ]:
!pip install medpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 40.0 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.5.2-cp310-cp310-linux_x86_64.whl size=762840 sha256=ff4ade2fa5087b9b1bacb08cdfee651a1de71a21e18cc9af145a8457b51014a7
  Stored in directory: /root/.cache/pip/wheels/a1/b8/63/bdf557940ec60d1b8822e73ff9fbe7727ac19f009d46b5d175
Successfully built medpy


In [5]:
!python test_kfold.py

10 Turtle
Model loaded !
==== 1 models loaded ====
Test visualize: 100%|████████████████| 10/10 [00:02<00:00,  3.49img/s]
inference_time 0.2883608818054199
dice_5folds: [97.19]
iou_5folds: [94.68]
dice: 97.19+nan
iou: 94.68+nan


### Segmentation Results visualized

### You can get the Dice and IoU scores and the visualization results `in the visualized folder`

### Here is the table containing all the results from three parts
| Metrics   | Head  | Carapace | Flippers | 
|-|-|-|-|
|**DICE**|`87.49%`|`96.81%`|`88.58%`|
|**mIoU**|`80.87%`|`94.51%`|`81.63%`|

In [7]:
import os
from PIL import Image
import matplotlib.pyplot as plt

image_folder = "./Turtle_visualize_test"
image_folder_F = "./Turtle_F_visualize_test"
image_folder_H = "./Turtle_H_visualize_test"

image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
image_files_F = sorted([f for f in os.listdir(image_folder_F) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
image_files_H = sorted([f for f in os.listdir(image_folder_H) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])


In [ ]:

# Print all segmented Carapace
if not image_files:
    print("No images in this folder!")
else:
    plt.figure(figsize=(10, len(image_files) * 3))

    for i, image_file in enumerate(image_files):
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)
        
        plt.subplot(len(image_files) // 2 + len(image_files) % 2, 2, i + 1)
        plt.imshow(image)
        plt.axis("off")
        plt.title(image_file)
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Print all segmented Flippers
if not image_files_F:
    print("No images in this folder!")
else:
    plt.figure(figsize=(10, len(image_files_F) * 3))

    for i, image_file in enumerate(image_files_F):
        image_path = os.path.join(image_folder_F, image_file)
        image = Image.open(image_path)
        
        plt.subplot(len(image_files_F) // 2 + len(image_files_F) % 2, 2, i + 1)
        plt.imshow(image)
        plt.axis("off")
        plt.title(image_file)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Print all segmented Head
if not image_files_H:
    print("No images in this folder!")
else:
    plt.figure(figsize=(10, len(image_files_H) * 3))

    for i, image_file in enumerate(image_files_H):
        image_path = os.path.join(image_folder_H, image_file)
        image = Image.open(image_path)
        
        plt.subplot(len(image_files_H) // 2 + len(image_files_H) % 2, 2, i + 1)
        plt.imshow(image)
        plt.axis("off")
        plt.title(image_file)
    
    plt.tight_layout()
    plt.show()